In [1]:
!pip install pymysql

In [2]:
!pip install pykrx

    high ,#고가
    low ,#저가
    end_close #종가
    trading_volume #거래량
    transaction_amount  #거래대금
    end_rate_change #등락율
    institutional_total #기관 합계
    other_corporations #기타 법인
    individual #개인
    foreigner_total # 외국인 합계
    short_selling ,#공매도
    short_buying # 공매도 매수
    short_importance #공매도 비중

In [8]:
import joblib
import pymysql
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
import platform
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from keras.backend import clear_session
from sklearn.metrics import classification_report
from pykrx import stock
import warnings
from keras.models import load_model

In [5]:

conn = pymysql.connect(host='localhost', user='root', password='1234', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)
stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)

# 총합 정확도 예측을 위한 변수
total = 0

#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 100
count=0
try:
    # for문에 
    for gam in [[10,5],[30,10]]:
        stock_name = []
        stock_short_acc= []
        stock_long_acc = []
        for i in range(0,stock_n):

            curs = conn.cursor()
            sql = "SELECT * FROM StockData WHERE stock_code ='{}'".format(str(stock_code_list['stock_code'][i]))
            # 코드 앞에서 A 지우고 주식 이름 불러오기
            name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))
            curs.execute(sql)
            data = curs.fetchall()

            df = pd.DataFrame(data)

            # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
            df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
            df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
            df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
            df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
            df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
            df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
            df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
            df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
            df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
            df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
            df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)

            df = df.fillna(method='bfill')
            df = df.fillna(method='ffill')

            df['1yd_end_close'] = df['end_close'].shift(1)
            df['1yd_trading'] = df['trading_volume'].shift(1)
            df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
            df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
            df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
            df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
            df['120일 평균 종가'] = df['end_close'].rolling(120).mean()

            df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
            df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
            df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
            df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
            df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()

            pd.set_option('mode.chained_assignment',  None)

            # X1 일 뒤 X2일 평균 종가
            X1 = gam[0]
            X2 = gam[1]
            df['X1일 뒤 종가'] = df['end_close'].shift(-X1)
            df['X1일 뒤 X2일 평균 종가'] = 0
            for i in range(0, df.shape[0]-X2):
                for j in range(0,X2):
                    df['X1일 뒤 X2일 평균 종가'][i] = df['X1일 뒤 X2일 평균 종가'][i] + df['X1일 뒤 종가'][i+j]

            df['X1일 뒤 X2일 평균 종가'] = df['X1일 뒤 X2일 평균 종가']/X2

            df['X1일 뒤 X2일 평균 종가 변화량'] = df['X1일 뒤 X2일 평균 종가'] - df['end_close']

            # 분류 모델 쓸 때 0,1로 라벨링 하기
            df['X1일 뒤 X2일 평균 종가 변화량'] = df['X1일 뒤 X2일 평균 종가 변화량'].apply(lambda x: 1 if x>0 else 0)

            df.dropna(inplace=True)
            sql2 = "SELECT * FROM MacroeconomicIndicators"
            curs.execute(sql2)
            data2 = curs.fetchall()

            df2 = pd.DataFrame(data2)

            df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
            df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
            df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
            df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()

            df2.dropna(inplace=True)

            df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')

            #merge하면서 생기는 id 행들 제거
            df_m.drop(columns=['id_x','id_y'],inplace=True)

            # 컬럼 제작
            df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
            df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
            df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
            df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
            df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
            df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
            df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
            df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
            df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
            df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

            df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
            df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
            df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
            df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
            df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

            df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
            df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
            df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
            df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']

            target = ['X1일 뒤 X2일 평균 종가 변화량']
            f_col = ['start_open', 'high', 'low', 'end_close',
                   'trading_volume', 'transaction_amount', 'end_rate_change',
                   'institutional_total', 'other_corporations', 'individual',
                   'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
                   '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
                   '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
                   '60일 평균 거래량', '120일 평균 거래량', 'kospi','inflation', 'treasury_bonds', 
                     'tightening','normality', 'powell', 'dispute', 'japan', 'volume', 'chairman',
                     'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                     'volatility', 'early_stage', 'decline', 'learning_result',
                   'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
                   '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
                   '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
                   '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
                   '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
                   '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
                   '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
                   '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                     '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
            X=df_m[f_col]
            Y=df_m[target]
            
            '''
            XGB코드
            '''
            X=df_m[f_col]
            Y=df_m[target]
            
            scaler = MinMaxScaler()
            X_s = scaler.fit_transform(X)
            X_s = pd.DataFrame(X_s,columns=X.columns)
            TEST_SIZE = 180
            WINDOW_SIZE = 10

            train, test, train_y, test_y = train_test_split(X_s, Y, test_size=0.2,shuffle=False)

            def make_dataset(data, label, window_size=10):
                feature_list = []
                label_list = []
                for i in range(len(data) - window_size):
                    feature_list.append(np.array(data.iloc[i:i+window_size]))
                    label_list.append(np.array(label.iloc[i+window_size]))
                return np.array(feature_list), np.array(label_list)

            train_feature = train
            train_label = train_y

            train_feature, train_label = make_dataset(train_feature, train_label, 10)
            x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.05)
            test_feature = test
            test_label = test_y
            test_feature, test_label = make_dataset(test_feature, test_label, 10)

            clear_session()
            model = Sequential()
            model.add(LSTM(16, 
                           input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                           activation='relu', 
                           return_sequences=False)
                      )

            model.add(Dense(1,activation='sigmoid'))


            import os

            model.compile(loss='mean_squared_error', optimizer='adam')
            early_stop = EarlyStopping(monitor='val_loss', patience=2)

            model_path = 'model'
            filename = os.path.join(model_path, 'tmp_checkpoint.h5')
            checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

            history = model.fit(x_train, y_train, 
                                                epochs=200, 
                                                batch_size=16,
                                                verbose=0,
                                                validation_data=(x_valid, y_valid), 
                                                callbacks=[early_stop, checkpoint])
            model.load_weights(filename)
            pred = model.predict(test_feature)
            pred=list(map(lambda x : 1 if x>0.5 else 0,pred))
            warnings.filterwarnings('ignore')
            count+=1

            print("AI 예측 정확도 - " + name + ":",f1_score(pred, test_label, average='macro'))
            

            total = total + f1_score(pred, test_label, average='macro')
            print("종목 총합 정확도 : ",(total/count))
            
            stock_short_acc=[]
            stock_long_acc=[]
            if gam[0] ==10:
                model.save('LSTM_short/'+name+'_model.h5')
                stock_name.append(f1_score(test_label, pred, average='macro'))
                stock_short_acc.append(f1_score(test_label, pred, average='macro'))
            else:
                model.save('LSTM_long/'+name+'_model.h5')
                stock_long_acc.append(f1_score(test_label, pred, average='macro'))


finally:
    conn.close()


    

AI 예측 정확도 - LG에너지솔루션: 0.0
종목 총합 정확도 :  0.0
AI 예측 정확도 - 카카오페이: 0.25925925925925924
종목 총합 정확도 :  0.12962962962962962
AI 예측 정확도 - F&F: 0.808888888888889
종목 총합 정확도 :  0.3560493827160494
AI 예측 정확도 - SK스퀘어: 1.0
종목 총합 정확도 :  0.5170370370370371
AI 예측 정확도 - LG에너지솔루션: 0.0
종목 총합 정확도 :  0.4136296296296297
AI 예측 정확도 - 카카오페이: 0.0
종목 총합 정확도 :  0.34469135802469136
AI 예측 정확도 - F&F: 0.5052083333333334
종목 총합 정확도 :  0.36762235449735453
AI 예측 정확도 - SK스퀘어: 0.45454545454545453
종목 총합 정확도 :  0.37848774200336704


In [32]:
df_acc = pd.DataFrame({'stock_name':stock_name,
                     'stock_short_acc':stock_short_acc,
                     'stock_long_acc':stock_long_acc})

df_acc.to_csv("top100예측정확도.csv",encoding='cp949',index=False)

100

100

0

In [13]:
import datetime

#오늘 날짜를 표시 
dt_today = datetime.date.today() 

print(dt_today.strftime('%Y-%m-%d'))
#2022-05-10


2022-12-26


In [1]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)
stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)


#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 100

try:
    # for문에 
    for i in range(0,stock_n):

        curs = conn.cursor()
        sql = "SELECT * FROM StockData WHERE (data_time BETWEEN '2022-06-01' AND '2022-12-08') AND (stock_code ='{}')".format(str(stock_code_list['stock_code'][i]))
        # 코드 앞에서 A 지우고 주식 이름 불러오기
        name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))
        curs.execute(sql)
        data = curs.fetchall()
        
        df = pd.DataFrame(data)
        
        df = df[-130:]
        
        # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
        df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
        df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
        df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
        df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
        df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
        df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
        df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
        df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
        df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)
          
        df = df.fillna(method='bfill')
        df = df.fillna(method='ffill')
    
        df['1yd_end_close'] = df['end_close'].shift(1)
        df['1yd_trading'] = df['trading_volume'].shift(1)
        df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
        df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
        df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
        df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
        df['120일 평균 종가'] = df['end_close'].rolling(120).mean()
    
        df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
        df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
        df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
        df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
        df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()
    
        pd.set_option('mode.chained_assignment',  None)

        df.dropna(inplace=True)
        
        sql2 = "SELECT * FROM MacroeconomicIndicators WHERE (date_time BETWEEN '2022-06-01' AND '2022-12-08')"
        curs.execute(sql2)
        data2 = curs.fetchall()

        df2 = pd.DataFrame(data2)
        
        df2 = df2[-130:]

        df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
        df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
        df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
        df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()
    
        df2.dropna(inplace=True)
        
        df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')
        
        #merge하면서 생기는 id 행들 제거
        df_m.drop(columns=['id_x','id_y'],inplace=True)
                                            
        # 컬럼 제작
        df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
        df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
        df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
        df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
        df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
        df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
        df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
        df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
        df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
        df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

        df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
        df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
        df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
        df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
        df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

        df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
        df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
        df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
        df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']
        
        f_col = ['start_open', 'high', 'low', 'end_close',
               'trading_volume', 'transaction_amount', 'end_rate_change',
               'institutional_total', 'other_corporations', 'individual',
               'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
               '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
               '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
               '60일 평균 거래량', '120일 평균 거래량', 'kospi', 
                 'inflation', 'treasury_bonds', 'tightening','normality', 
                 'powell', 'dispute', 'japan', 'volume', 'chairman',
                 'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                 'volatility', 'early_stage', 'decline', 'learning_result',
               'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
               '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
               '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
               '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
               '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
               '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
               '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
               '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                 '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
        X=df_m[f_col]
        '''
        XGB코드
        '''
        test_x = df_m[f_col]
        
        scaler = MinMaxScaler()
        X_s = scaler.fit_transform(test_x)
        X_s = pd.DataFrame(X_s,columns=test_x.columns)
        TEST_SIZE = 180
        WINDOW_SIZE = 10

        def make_dataset(data, window_size=10):
            feature_list = []
            for i in range(len(data) - window_size):
                feature_list.append(np.array(data.iloc[i:i+window_size]))
            return np.array(feature_list)
        test_feature= make_dataset(X_s, 10)
        
        model = load_model('LSTM_long/'+name+'_model.h5')
        pred = model.predict(test_feature)
        
        print()
        print(name + "의 " + str(df_m['data_time'][0])+" 예측 결과 = ",round(pred[0][0],5))
finally:
    conn.close()
    


NameError: name 'pymysql' is not defined

In [26]:
df_m

,stock_code,data_time,start_open,high,low,end_close,trading_volume,transaction_amount,end_rate_change,institutional_total,...,120일 평균 종가 대비 당일 종가 비율,5일평균거래량대비당일거래량,10일평균거래량대비당일거래량,20일평균거래량대비당일거래량,60일평균거래량대비당일거래량,120일평균거래량대비당일거래량,코스피지수5일평균종가대비당일종가비율,코스피지수20일평균종가대비당일종가비율,코스피지수60일평균종가대비당일종가비율,코스피지수120일평균종가대비당일종가비율
0,A402340,2022-11-24,37600.0,37700.0,37150.0,37450.0,178901.0,6.684130e+09,0.00,5.713130e+08,...,0.928474,0.590626,0.568491,0.462099,0.451438,0.463137,1.006436,1.017300,1.047498,1.023775
1,A402340,2022-11-25,37500.0,37500.0,36950.0,37100.0,161473.0,5.994170e+09,-0.93,5.713130e+08,...,0.921520,0.588364,0.584930,0.436889,0.410906,0.419574,1.005554,1.012280,1.045927,1.023111
2,A402340,2022-11-28,36950.0,36950.0,36100.0,36450.0,270040.0,9.817490e+09,-1.75,5.713130e+08,...,0.907347,1.124469,0.994054,0.731833,0.687960,0.709708,0.994270,0.997619,1.033547,1.011621
3,A402340,2022-11-29,36000.0,36850.0,36000.0,36650.0,206096.0,7.533130e+09,0.55,5.713130e+08,...,0.913956,0.956687,0.787656,0.564569,0.532932,0.549242,1.002314,1.005979,1.044616,1.022864
4,A402340,2022-11-30,36450.0,37800.0,36200.0,37450.0,1487470.0,5.566810e+10,2.18,5.713130e+08,...,0.935539,3.228045,3.854215,3.609203,3.686645,3.870408,1.013882,1.019302,1.060986,1.039875
5,A402340,2022-12-01,37950.0,38200.0,37400.0,37500.0,365908.0,1.383150e+10,0.13,5.713130e+08,...,0.938546,0.734464,0.913516,0.953732,0.907002,0.952654,1.013678,1.019150,1.063588,1.043482
6,A402340,2022-12-02,37500.0,37700.0,36900.0,36900.0,253146.0,9.407330e+09,-1.60,1.965450e+09,...,0.925208,0.490088,0.640085,0.687114,0.629469,0.659003,0.995362,0.998684,1.043840,1.024913
7,A402340,2022-12-05,36800.0,37350.0,36250.0,36300.0,346288.0,1.264450e+10,-1.63,1.965450e+09,...,0.911440,0.651185,0.897200,0.952959,0.860638,0.903825,0.988332,0.991559,1.037335,1.018898
8,A402340,2022-12-06,35950.0,36250.0,35650.0,35650.0,402108.0,1.441400e+10,-1.79,1.965450e+09,...,0.896479,0.704237,1.022641,1.093533,0.998828,1.049712,0.980869,0.980956,1.025996,1.008234
9,A402340,2022-12-07,35550.0,35650.0,35150.0,35200.0,386858.0,1.368650e+10,-1.26,1.965450e+09,...,0.886399,1.102594,0.953254,1.047703,1.046549,1.010208,0.983863,0.977547,1.021569,1.004101
